In [138]:
import tensorflow as tf
import numpy as np

In [139]:
input__ = np.array([np.random.randint(0,20) for i in range(40)]).reshape(4,10)
input__

array([[17, 15,  8,  9, 18,  9,  4,  4, 19,  8],
       [10,  2, 16,  8,  2, 11, 14,  3,  8,  6],
       [ 3, 12,  0,  4,  5,  3, 14,  8,  9,  1],
       [ 2, 16,  9,  7,  2,  3, 16, 19, 18, 14]])

In [140]:
# input__ = np.random.randn(40)
input_ = tf.keras.Input(shape=(10))
embedding_layer = tf.keras.layers.Embedding(20,4)
linear = tf.keras.layers.Dense(20)
linear2 = tf.keras.layers.Dense(4)
bn = tf.keras.layers.BatchNormalization()
fc = tf.keras.layers.Dense(20)



In [141]:
fc.set_weights(tf.transpose(embedding_layer.weights))

In [142]:
out = embedding_layer(input_)

In [143]:
out = linear(out)

In [144]:
out

<KerasTensor: shape=(None, 10, 20) dtype=float32 (created by layer 'dense_9')>

In [145]:
out = linear2(out)
out

<KerasTensor: shape=(None, 10, 4) dtype=float32 (created by layer 'dense_10')>

In [146]:
out = bn(out)

In [147]:
out = fc(out)

In [148]:
out

<KerasTensor: shape=(None, 10, 20) dtype=float32 (created by layer 'dense_11')>

In [149]:

model = tf.keras.Model(inputs= input_, outputs = out)
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

optimizer = tf.keras.optimizers.Adam()


loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [150]:
out_ = model(input__)
out_

<tf.Tensor: shape=(4, 10, 20), dtype=float32, numpy=
array([[[ 4.94249025e-03,  7.77023938e-03,  2.04750840e-02,
          6.07669773e-03, -2.70922966e-02,  1.11365668e-03,
          5.38740726e-03,  2.83803698e-03, -7.20122503e-03,
          4.44067083e-03,  6.47371961e-03, -1.38952378e-02,
          5.38109336e-03,  3.68611119e-03, -7.98033737e-03,
          7.40007148e-04, -6.36450946e-03,  7.72999774e-04,
          9.12677217e-03,  1.89671908e-02],
        [ 6.59193983e-03,  4.76926239e-03,  1.51989497e-02,
          6.07170351e-03, -1.95021853e-02,  5.84156206e-03,
          3.47818015e-04,  8.10763706e-03, -2.12085899e-03,
          6.27612509e-03,  6.23651175e-03, -1.26551520e-02,
         -1.51763647e-03,  7.19563803e-03, -3.41235194e-03,
         -5.36223780e-03, -4.64075757e-03,  4.23644856e-03,
          4.59520984e-03,  1.20826410e-02],
        [-2.32176180e-03, -1.16665941e-02, -2.62407791e-02,
         -9.43527184e-03,  3.89756821e-02,  7.76718324e-03,
         -1.4260652

In [205]:
with tf.GradientTape() as tape:
    out_ = model(input__)
    # print(out_)
    losses = loss_function(np.array([[2,5,3,1,6,2,4,1,1,3]]*4),out_)
# print(losses)
gradients = tape.gradient(losses,model.trainable_variables)
gradients

 <tf.Tensor: shape=(4, 20), dtype=float32, numpy=
 array([[ 2.4372341e-04, -1.3293626e-04, -2.3965424e-04,  5.6887651e-04,
         -7.3585782e-04,  3.4357901e-04,  5.2541902e-04, -6.3039581e-05,
          8.8699511e-07,  2.2093163e-04, -3.2565789e-04, -7.9748213e-05,
         -2.8594749e-04, -9.6308324e-04, -1.4948998e-04,  6.0544879e-04,
          1.7570093e-04,  2.0209314e-04,  4.3699740e-05,  1.6014808e-04],
        [-5.1649159e-04, -3.9163078e-04,  6.3652382e-04, -1.8228007e-03,
          2.1052270e-03, -2.5885884e-04, -3.0025741e-04,  5.3849770e-04,
         -4.1918774e-04, -8.7376719e-04,  2.7035072e-04,  4.5933813e-04,
          9.2289620e-04,  1.8602628e-03, -2.7231171e-04, -3.2123478e-04,
          3.2642507e-04,  2.0011666e-04, -8.7443914e-04,  3.3753073e-05],
        [-5.4585107e-04, -6.9571019e-04,  4.8129083e-04, -1.5785912e-03,
          1.6907130e-03, -5.9790054e-05,  6.3265004e-04,  7.7247160e-04,
         -6.4752443e-04, -7.6743070e-04, -6.2782980e-05,  5.1510253e-04,

In [206]:
model.trainable_variables

[<tf.Variable 'embedding_3/embeddings:0' shape=(20, 4) dtype=float32, numpy=
 array([[ 1.10999653e-02, -3.83384340e-02,  2.41710898e-02,
         -3.21554281e-02],
        [-5.94715402e-03,  2.42126454e-02, -3.60523071e-03,
         -1.04534943e-02],
        [-1.67130269e-02,  2.84536905e-03, -4.40495834e-02,
         -3.73688005e-02],
        [ 1.89467873e-02, -4.38858829e-02, -4.87811193e-02,
          2.76379120e-02],
        [ 1.32939145e-02, -4.59048934e-02,  2.42201965e-02,
          1.94228441e-02],
        [ 4.85668480e-02,  4.83210944e-02,  2.30000727e-02,
          3.15383635e-02],
        [-3.22231688e-02, -1.08093796e-02,  1.67416446e-02,
          1.73198972e-02],
        [-1.05799017e-02, -3.71488445e-02,  2.92785075e-02,
          4.22924794e-02],
        [ 3.65821621e-03,  4.38979827e-02,  4.48153913e-02,
         -3.79259735e-02],
        [ 5.06082065e-02, -3.31536122e-03,  1.05103217e-02,
         -4.32637259e-02],
        [-3.95310372e-02, -4.42803428e-02,  2.4528460

In [207]:
optimizer = tf.keras.optimizers.Adam()

In [208]:
optimizer.apply_gradients(zip(gradients, model.trainable_variables))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [209]:
bn.weights

[<tf.Variable 'batch_normalization_4/gamma:0' shape=(4,) dtype=float32, numpy=array([1.0019723, 0.998003 , 1.0019753, 1.0019968], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/beta:0' shape=(4,) dtype=float32, numpy=array([ 0.00199977, -0.00199971, -0.0019998 ,  0.00199994], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_variance:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>]

In [162]:
bn.weights

[<tf.Variable 'batch_normalization_4/gamma:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/beta:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_variance:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>]

In [108]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 10, 4)             80        
                                                                 
 dense_6 (Dense)             (None, 10, 20)            100       
                                                                 
 dense_7 (Dense)             (None, 10, 4)             84        
                                                                 
 batch_normalization_2 (Batc  (None, 10, 4)            16        
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 10, 20)            100       
                                                           

In [75]:
embedding_layer.weights,tf.transpose(fc.weights[0])

([<tf.Variable 'embedding_2/embeddings:0' shape=(20, 4) dtype=float32, numpy=
  array([[ 0.03296108,  0.04589659,  0.01771945,  0.02754359],
         [ 0.03289864,  0.03147545,  0.02349153, -0.0487397 ],
         [-0.04067399,  0.01326983, -0.0385762 , -0.03243945],
         [-0.00946356,  0.04343499, -0.01105589, -0.03926749],
         [-0.04057728, -0.0490714 , -0.00203189, -0.0314842 ],
         [ 0.04064996, -0.0054789 ,  0.03747949,  0.01983969],
         [-0.0333234 , -0.00152744, -0.00644578, -0.00760757],
         [-0.02937509, -0.02008191,  0.00615846, -0.0236658 ],
         [-0.00405482, -0.03840353, -0.01952491, -0.00561755],
         [ 0.01435752,  0.02569569,  0.01444266, -0.03265146],
         [-0.04589455,  0.03572767, -0.03221891,  0.00871722],
         [-0.04354692,  0.01300658, -0.01216329, -0.01719166],
         [-0.02076933, -0.00643247,  0.01087897, -0.0092754 ],
         [-0.01870356,  0.02524734,  0.02369357,  0.02996296],
         [ 0.04617624, -0.04903411,  0.0

In [54]:
fc.set_weights()
# fc.set_weights(tf.transpose(embedding_layer.weights))

TypeError: Layer.set_weights() missing 1 required positional argument: 'weights'

In [57]:
tf.transpose(embedding_layer.weights[0])

<tf.Tensor: shape=(4, 20), dtype=float32, numpy=
array([[ 3.4607422e-02, -4.5584142e-02, -2.9752739e-03,  3.1040575e-02,
        -2.3792982e-02, -4.8548926e-02, -4.9865842e-03,  4.0170763e-02,
         4.5857940e-02, -1.0359656e-02, -3.2776915e-02,  4.4368256e-02,
        -2.5764465e-02, -9.2743263e-03,  4.9272548e-02, -3.3403266e-02,
         1.9228052e-02, -6.6880696e-03, -4.4297624e-02, -5.3079017e-03],
       [ 4.0111732e-02,  9.2930682e-03, -1.0112107e-02, -1.6078293e-02,
        -4.4211566e-02, -1.3282977e-02,  4.9257424e-02,  4.9948122e-02,
        -5.2746423e-03, -2.0141412e-02,  4.3407012e-02,  2.8072421e-02,
         2.1639135e-02, -4.6937883e-02, -2.9105060e-03,  2.5935721e-02,
        -3.8762011e-02,  4.7923412e-02, -2.6015580e-02,  3.9625932e-02],
       [ 3.8148288e-02,  2.0635016e-03,  3.8257129e-03,  5.7753325e-03,
         4.3380167e-02, -3.9641868e-02,  1.5145317e-03,  1.5820097e-02,
         1.6975030e-03, -4.2086422e-02, -2.2760822e-02, -2.9350376e-02,
         9.05

In [38]:
tf.squeeze(tf.transpose(embedding_layer.weights))
# embedding_layer.weights

<tf.Tensor: shape=(4, 20), dtype=float32, numpy=
array([[ 3.4607422e-02, -4.5584142e-02, -2.9752739e-03,  3.1040575e-02,
        -2.3792982e-02, -4.8548926e-02, -4.9865842e-03,  4.0170763e-02,
         4.5857940e-02, -1.0359656e-02, -3.2776915e-02,  4.4368256e-02,
        -2.5764465e-02, -9.2743263e-03,  4.9272548e-02, -3.3403266e-02,
         1.9228052e-02, -6.6880696e-03, -4.4297624e-02, -5.3079017e-03],
       [ 4.0111732e-02,  9.2930682e-03, -1.0112107e-02, -1.6078293e-02,
        -4.4211566e-02, -1.3282977e-02,  4.9257424e-02,  4.9948122e-02,
        -5.2746423e-03, -2.0141412e-02,  4.3407012e-02,  2.8072421e-02,
         2.1639135e-02, -4.6937883e-02, -2.9105060e-03,  2.5935721e-02,
        -3.8762011e-02,  4.7923412e-02, -2.6015580e-02,  3.9625932e-02],
       [ 3.8148288e-02,  2.0635016e-03,  3.8257129e-03,  5.7753325e-03,
         4.3380167e-02, -3.9641868e-02,  1.5145317e-03,  1.5820097e-02,
         1.6975030e-03, -4.2086422e-02, -2.2760822e-02, -2.9350376e-02,
         9.05

In [48]:
fc.weights

[<tf.Variable 'dense_2/kernel:0' shape=(4, 20) dtype=float32, numpy=
 array([[ 2.46511459e-01,  6.50435686e-02,  4.36426997e-01,
         -4.50643063e-01,  1.38709188e-01, -2.42868662e-02,
         -3.37507963e-01,  1.31726265e-04, -4.94050622e-01,
         -1.70221925e-01,  3.46983552e-01,  2.26798177e-01,
         -1.43355250e-01,  4.17590141e-03, -3.06314588e-01,
         -5.67567348e-03, -3.17600012e-01, -1.64951682e-01,
          4.70630646e-01,  3.23259830e-03],
        [-5.37379980e-02,  1.79540038e-01,  2.03818917e-01,
          2.76816845e-01,  3.67062807e-01,  1.65161371e-01,
          4.92871046e-01, -4.16972280e-01, -3.18400383e-01,
         -1.60263777e-02, -1.65850282e-01, -3.86006832e-01,
          2.40555882e-01,  1.23026371e-02, -4.25285101e-01,
          3.85877609e-01, -2.35045552e-01,  2.21004844e-01,
          1.46369219e-01, -2.51801014e-01],
        [-2.01126337e-02,  2.87664175e-01,  3.49067092e-01,
          4.58373427e-01, -1.04699135e-02, -2.84510016e-01,
   

In [126]:
bn.weights

[<tf.Variable 'batch_normalization_2/gamma:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/beta:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/moving_mean:0' shape=(4,) dtype=float32, numpy=
 array([-1.1790880e-05, -1.5876224e-04, -6.4277359e-05,  5.8590922e-05],
       dtype=float32)>,
 <tf.Variable 'batch_normalization_2/moving_variance:0' shape=(4,) dtype=float32, numpy=array([0.9801039 , 0.98010284, 0.98011065, 0.9801077 ], dtype=float32)>]

In [136]:
with tf.GradientTape() as what:
    batch_out = bn(np.array([[[1,2,3,4],[100,200,300,400]]]))
batch_out

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[  0.9995004,   1.9990008,   2.9985013,   3.9980016],
        [ 99.95004  , 199.90009  , 299.85013  , 399.80017  ]]],
      dtype=float32)>

In [135]:
bn = tf.keras.layers.BatchNormalization()

In [151]:
bn.weights

[<tf.Variable 'batch_normalization_4/gamma:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/beta:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_variance:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>]

In [172]:
np.dot(np.array([[1,2,3],[4,5,6]]),np.array([[1,1,1],[0,0,0]]).T)

array([[ 6,  0],
       [15,  0]])

In [179]:
np.dot(np.ones((2,3,4,5)),np.ones((2,3,5,4))).shape
# np.dot(np.ones((2,3,4,5)),np.ones((2,3,5,4)))

(2, 3, 4, 2, 3, 4)

In [181]:
np.matmul(np.array([[1,2,3],[4,5,6]]),np.array([[1,2,3],[4,5,6]]).T)

array([[14, 32],
       [32, 77]])

In [ ]:
np.dot(np.ones((2,3,4,5)), np.ones((2,3,5,4)))

In [198]:
np.dot(np.ones((2,4)),np.ones((2,4,3)))

array([[[4., 4., 4.],
        [4., 4., 4.]],

       [[4., 4., 4.],
        [4., 4., 4.]]])